In [1]:
%matplotlib notebook
%reload_ext autoreload
%autoreload 2

import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import ipyvolume as ipv

sys.path.append('../')
sys.path.append('../paretoviz')
from paretoviz import plotting as pplt
from paretoviz import decorator as dcor
from paretoviz.utils import fmt
from paretoviz.utils import vectorops as vops

x, y, z = np.random.random((3, 100))

ipv.figure()

ipv.style.axes_off()
ipv.style.box_off()

ipv.scatter(x, y, z, size = 2, marker = "sphere")

ipv.show()

In [8]:
rawfpath = "../data/knee/knee-3d.out"

# get the path and filename from the rawfpath
path, rawfile = os.path.split(rawfpath)
# get the prefix
prefix = rawfile.split('.')[0]

# do visualization based on distance from the centroid?
docentroid = False

# load the normalized points
points = fmt.load(os.path.join(path, prefix + "-norm.out"))

# load the CV values
cvfpath = os.path.join(path, prefix + "-cv.out")
if not docentroid and os.path.exists(cvfpath):
    cv = [v[0] if len(v) == 1 else v for v in fmt.load(cvfpath)]
    [low, up] = vops.get_bound(cv)
    cv = vops.normalize(cv, low, up)
    color = dcor.recolor_by_cv(cv)
else:
    color = dcor.recolor_by_centroid(points)

# load the normalized trade-off values
mufpath = os.path.join(path, prefix + "-norm-mu.out")
knee_idx = None
if os.path.exists(mufpath):
    mu = [v[0] if len(v) == 1 else v for v in fmt.load(mufpath)]
    # resize the points w.r.t. trade-offs
    size = dcor.rescale_by_tradeoff(mu)
    (color, knee_idx) = dcor.recolor_by_tradeoff(size, color)

# load the class labels
classfpath = os.path.join(path, prefix + "-class.out")
if os.path.exists(classfpath):
    labels = [v[0] if len(v) == 1 else v for v in \
            fmt.load(classfpath, dtype = dcor.dtypes[prefix])]
    color = dcor.recolor_by_labels(labels, dtype = dcor.dtypes[prefix])
    size = [5.0 for _ in range(len(points))]

# alpha values
alpha = [0.2, 0.8] # alpha for plots with knee
# alpha = [1.0, 1.0] # alpha for general case

palette_coords = fmt.load(os.path.join(path, prefix + "-norm-palette-invsc.out"))
print(palette_coords[0])

data = np.array(palette_coords)
print(data[0,:])
print(data[:,0])

size = [s*0.075 for s in size]

ipv.figure()

ipv.style.axes_off()
ipv.style.box_off()
ipv.selector_default()

ipv.scatter(data[:,0], data[:,1], data[:,2], size = size, color = color, alpha = 0.01, marker = "sphere")

ipv.show()

[-0.1433, 0.2225, -2.0]
[-0.1433  0.2225 -2.    ]
[-0.1433  0.2302  0.0864 ... -0.0166  0.0558  0.1273]


Output()